# A Quick Tutorial for Implicit Deep Learning

This tutorial introduces the **Implicit Deep Learning** (IDL) framework using the `idl` package in 3 main parts:

1. **A Simple Example**
    - Implicit Model
    - Implcit RNN
    - State-driven Implicit Model (SIM)
3. **Custom Activation for Implicit model**
4. **Implicit model as a layer**

## 1. A Simple Example

This section provides a quick guide on how to use our package. With just a few lines of code, you can get started effortlessly.

Before proceeding, please ensure you have installed the required packages by following the [installation](https://github.com/HoangP8/Implicit-Deep-Learning?tab=readme-ov-file#installation) instructions.

#### 1a. `ImplicitModel`

`ImplicitModel` is the most fundamental implicit model. For details on its parameters and the underlying intuition, please refer to the [documentation](link).

In this example, we demonstrate how to use the model for a simple regression task.

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from idl import ImplicitModel

torch.manual_seed(42) # fix seed

# Random input data
x = torch.randn(5, 64)  # (batch_size=5, input_dim=64)

# Random ground truth values
y = torch.randn(5, 10)  # (batch_size=5, output_dim=10)

# Initialize the model
model = ImplicitModel(input_dim=64,
                      output_dim=10, 
                      hidden_dim=128)

# Define MSE loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    optimizer.zero_grad() 
    output = model(x)  # Forward pass
    loss = criterion(output, y)  # Compute MSE loss
    loss.backward() 
    optimizer.step()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")
        
# Inference step
model.eval()  
with torch.no_grad():  
    x_test = torch.randn(1, 64)  
    y_pred = model(x_test)  
    print(f"Inference result: \n {y_pred}")

Epoch [1/10], Loss: 0.7182
Epoch [2/10], Loss: 0.3191
Epoch [3/10], Loss: 0.1070
Epoch [4/10], Loss: 0.1025
Epoch [5/10], Loss: 0.0644
Epoch [6/10], Loss: 0.0446
Epoch [7/10], Loss: 0.0357
Epoch [8/10], Loss: 0.0261
Epoch [9/10], Loss: 0.0170
Epoch [10/10], Loss: 0.0130
Inference result: 
 tensor([[ 0.1920,  0.2060, -0.3569, -0.2839, -0.1086,  0.3988, -0.2308, -0.2174,
         -0.0737,  0.0042]])


The `ImplicitModel` has its forward and backward passes **fully packaged**, ensuring that the training and inference steps work **as normal**, with no additional modifications required. You only need to define the model with the appropriate `input_dim`, `output_dim`, and `hidden_dim`, and use it just like any other model.

#### 1b. `ImplicitRNN`

`ImplicitRNN` uses an implicit layer to define recurrence within a standard RNN framework. For more details, please refer to the [documentation](link).

Its usage is very similar to `ImplicitModel`. Below, we provide an example where the model learns to predict a single output from an input sequence in a simple regression task.

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from idl import ImplicitRNN

# Fix seed for reproducibility
torch.manual_seed(42)

# Random input sequence data
x = torch.randn(50, 20, 1)  # (batch_size=50, seq_len=20, input_dim=1)

# Random ground truth sequence values
y = torch.randn(50, 1)  # (batch_size=50, output_dim=1)

# Initialize the ImplicitRNN model
model = ImplicitRNN(input_dim=1, output_dim=1, hidden_dim=10, implicit_hidden_dim=10)

# Define MSE loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loops
num_epochs = 10
for epoch in range(num_epochs):
    optimizer.zero_grad()
    
    output = model(x)  # Forward pass
    loss = criterion(output, y)  # Compute MSE loss
    loss.backward()
    optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# Inference step
model.eval()
with torch.no_grad():
    x_test = torch.randn(1, 20, 1)
    y_pred = model(x_test)
    print(f"Inference result: {y_pred}")

Epoch [1/10], Loss: 1.0385
Epoch [2/10], Loss: 1.0199
Epoch [3/10], Loss: 1.0032
Epoch [4/10], Loss: 0.9882
Epoch [5/10], Loss: 0.9739
Epoch [6/10], Loss: 0.9594
Epoch [7/10], Loss: 0.9435
Epoch [8/10], Loss: 0.9255
Epoch [9/10], Loss: 0.9043
Epoch [10/10], Loss: 0.8801
Inference result: tensor([[-0.5701]])


#### 1c. `SIM`

## 2. Custom Activation for Implicit model
The default activation of the Implicit model is ReLU. To override the implicit function you wish to use, just simply replace the `phi` and `dphi` (gradient of activation) methods. Below is an example of SiLU activation.

In [ ]:
# ImplicitFunctionInf: function to ensure wellposedness of Implicit model
from idl import ImplicitModel, ImplicitFunctionInf 
import torch
import torch.nn as nn
import torch.optim as optim

class ImplicitFunctionInfSiLU(ImplicitFunctionInf):
    """
    An implicit function that uses the SiLU nonlinearity.
    """
    
    @staticmethod
    def phi(X):
        return X * torch.sigmoid(X)

    @staticmethod
    def dphi(X):
        grad = X.clone().detach()
        sigmoid = torch.sigmoid(grad)
        return sigmoid * (1 + grad * (1 - sigmoid))


# Initialize the model
model = ImplicitModel(input_dim=64,
                      output_dim=10, 
                      hidden_dim=128,
                      f=ImplicitFunctionInfSiLU)

# train model normally after

## Implicit model as a layer
Implicit Model can be integrated as a layer within larger models, allowing it to be trained as part of the overall network. The training process works normally, below is an example:


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from idl import ImplicitModel

# Define a larger model that includes ImplicitModel as a layer
class MLPWithImplicit(nn.Module):
    def __init__(self, input_dim, hidden_dim, implicit_hidden_dim, output_dim):
        super(MLPWithImplicit, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.implicit_layer = ImplicitModel(input_dim=hidden_dim, output_dim=output_dim, hidden_dim=implicit_hidden_dim)
        self.activation = nn.ReLU()

    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.implicit_layer(x)  # Pass through ImplicitModel
        return x


torch.manual_seed(42) # fix seed

# Random input data
x = torch.randn(5, 64)  # (batch_size=5, input_dim=64)

# Random ground truth values
y = torch.randn(5, 10)  # (batch_size=5, output_dim=10)

# Initialize the model
model = MLPWithImplicit(input_dim=64, hidden_dim=128, implicit_hidden_dim=64, output_dim=10)

# Define MSE loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    optimizer.zero_grad() 
    output = model(x)  # Forward pass
    loss = criterion(output, y)  # Compute MSE loss
    loss.backward() 
    optimizer.step()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")
        
# Inference step
model.eval()  
with torch.no_grad():  
    x_test = torch.randn(1, 64)  
    y_pred = model(x_test)  
    print(f"Inference result: \n {y_pred}")

Epoch [1/10], Loss: 0.7000
Epoch [2/10], Loss: 0.3326
Epoch [3/10], Loss: 0.1685
Epoch [4/10], Loss: 0.0587
Epoch [5/10], Loss: 0.0519
Epoch [6/10], Loss: 0.0429
Epoch [7/10], Loss: 0.0319
Epoch [8/10], Loss: 0.0247
Epoch [9/10], Loss: 0.0217
Epoch [10/10], Loss: 0.0204
Inference result: 
 tensor([[-0.7506, -0.1255,  0.1125, -0.7755,  0.6034,  1.0982,  0.3507,  0.2699,
          0.3629,  0.4228]])
